# Make a quick fit using astropy.modeling

## Authors
Rocio Kiman

## Learning Goals
* Know basic models in Astropy Modeling
* Be able to make a quick fit of your data
* Visualize the fit

## Keywords
Modeling, Fit 

## Summary
In this tutorial, we will become familiar with the models available in `astropy.modeling` and learn how to make a quick fit of our data.

### List of available models

* `Gaussian1D`
* `Trapezoid1D`
* `Polynomial1D`
* `Sine1D`
* `Linear1D`
* The [list](http://docs.astropy.org/en/stable/modeling/#module-astropy.modeling.functional_models) continues.

### List of available fitters

* `LinearLSQFitter()`       A class performing a linear least square fitting.
* `SLSQPLSQFitter()`        SLSQP optimization algorithm and least squares statistic.
* `LevMarLSQFitter()`       Levenberg-Marquardt algorithm and least squares statistic.
* `SimplexLSQFitter()`      Simplex algorithm and least squares statistic.

Check http://docs.astropy.org/en/stable/modeling/ for more information

### Imports

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from astropy.modeling import models, fitting
from astroquery.vizier import Vizier
# Make plots display in notebooks
%matplotlib inline 

## 1) Fit a Lineal model: Three steps to fit data using astropy.modeling

We are going to start with a lineal fit to real data. This data comes from the paper [Bhardwaj et al. 2017](https://ui.adsabs.harvard.edu/?#abs/2017A%26A...605A.100B). To get it, we are going to import it from [Vizier](http://vizier.u-strasbg.fr/viz-bin/VizieR) using [astroquery](http://astroquery.readthedocs.io/en/latest/vizier/vizier.html). You don't need internet for this step.

In [ ]:
catalog = Vizier.get_catalogs('J/A+A/605/A100')

This catalog has a lot of information, so we are going to put a special name to the arrays we need.

In [ ]:
period = np.array(catalog[0]['Period']) 
log_period = np.log10(period)
k_mag = np.array(catalog[0]['__Ksmag_'])
k_mag_err = np.array(catalog[0]['e__Ksmag_'])

Now we are going to plot this data, as any astronomer would do when they do research.

In [ ]:
plt.errorbar(log_period,k_mag,k_mag_err,fmt='k.')
plt.xlabel(r'$\log_{10}$(Period [days])')
plt.ylabel('Ks')

As we can see, this looks like a line. To probe it, we want to make a fit to the data. This is where `astropy.modeling` is useful. We are going to understand how in three simple lines we can make any fit we want. But let's start with the linear fit.

#### Let's start with the first step: Set up the model

First we need to choose which model we are going to fit to our data. As we said before, our data looks like a linear relation, so we are going to use a polynomial of degree 1. If you want, you can use `Linear1D` and compare the results.

In [ ]:
model = models.Polynomial1D(degree=1)

#### Second step: Set up the fitter

Second we are going to choose the fitter we want to use. This choise is basically which method we want to use to fit the model to the data. In this case we are going to use a the [*Linear Least Square Fitting*](https://www.mathworks.com/help/curvefit/least-squares-fitting.html).

In [ ]:
fitter = fitting.LinearLSQFitter() 

#### Third step: Fit the data

Finally, we give to our **fitter** (method to fit the data) the **model** and the **data** to perform the fit. Note that we are adding weights: This means that values with higher error will have smaller weight (less importance) in the fit, and the contrary for point with smaller error. This way of fitting is called *Weighted Least Squares* and you can find more information about it [here](https://www.mathworks.com/help/curvefit/least-squares-fitting.html).

In [ ]:
best_fit = fitter(model, log_period, k_mag, weights=1.0/k_mag_err**2)

If we check what is this new element we created: 

In [ ]:
print(best_fit)

We can see it has the coefficients for our fit were: `y = c0 + c1*x`.

And that's it!

We can easily evaluate the fit for our particular x axis by doing `best_fit(x)`.

In [ ]:
plt.errorbar(log_period,k_mag,k_mag_err,fmt='k.')
plt.plot(log_period, best_fit(log_period), color='g', linewidth=3)  
plt.xlabel(r'$\log_{10}$(Period [days])')
plt.ylabel('Ks')

**Conclusion:** Remember, with three lines you can have your fit:
    > model
    > fitter
    > perform fit

## 2) Fit a Polynomial model: Choose fitter wisely

For second example, lets fit a polynomial of degree more than 1. In this case, we are going to create fake data to make the fit. Note that we are adding gaussian noise to the data with the function `np.random.normal(0,2)` which gives a random number from a gaussian distribution with mean 0 and width 2.

In [ ]:
N = 100
x1 = np.linspace(0, 4, N)  # Makes an array from 0 to 4 of N elements
y1 = x1**3 - 6*x1**2 + 12*x1 - 9 
# Now we add some noise to the data
y1 = np.array([y_point + np.random.normal(0,2) for y_point in y1])  
sigma = 1.5
y1_err = np.ones(N)*sigma 

Let's plot it to see how it looks like:

In [ ]:
plt.errorbar(x1, y1, yerr=y1_err,fmt='k.')
plt.xlabel('$x_1$')  
plt.ylabel('$y_1$')

To fit this data lets remember the three steps: model, fitter and perform fit. 

In [ ]:
model_poly = models.Polynomial1D(degree=3)
fitter_poly = fitting.LinearLSQFitter() 
best_fit_poly = fitter_poly(model_poly, x1, y1, weights = 1.0/y1_err**2)

In [ ]:
print(best_fit_poly)

What if we don't want to use that fitter (method) to fit the data? Lets use the same model but with a different fitter: `SimplexLSQFitter`.

In [ ]:
fitter_poly_2 = fitting.SimplexLSQFitter()
best_fit_poly_2 = fitter_poly_2(model_poly, x1, y1, weights = 1.0/y1_err**2)

In [ ]:
print(best_fit_poly_2)

#### Compare results

One way to check which method is best is calculating the [*Reduce Chi Square Value*](https://en.wikipedia.org/wiki/Reduced_chi-squared_statistic). Lets define a function to do that because we are going to use it several times.

In [ ]:
def calc_reduce_chi_square(fit, x, y, yerr, N, n_free):
    '''
    fit (array) values for the fit
    x,y,yerr (arrays) data
    N total number of points
    n_free number of parameters we are fitting
    '''
    return 1.0/(N-n_free)*sum(((fit - y)/yerr)**2)

In [ ]:
reduce_chi_squared = calc_reduce_chi_square(best_fit_poly(x1), x1, y1, y1_err, N, 4)
print('Reduce Chi Squared with LinearLSQFitter: {}'.format(reduce_chi_squared))

In [ ]:
reduce_chi_squared = calc_reduce_chi_square(best_fit_poly_2(x1), x1, y1, y1_err, N, 4)
print('Reduce Chi Squared with SimplexLSQFitter: {}'.format(reduce_chi_squared))

As we can see, the *Reduce Chi Square* for the first fit is closer to one, which means this fit is better. Note that when we used the second fitter we got a warning saying that as the model is linear, we should use a linear method to fit the data (linear fitter) and that the iteration were not enough to make a good fit. 

In [ ]:
plt.errorbar(x1, y1, yerr=y1_err,fmt='k.')
plt.plot(x1, best_fit_poly(x1), color='r', linewidth=3, label='LinearLSQFitter()')  
plt.plot(x1, best_fit_poly_2(x1), color='g', linewidth=3, label='SimplexLSQFitter()')
plt.xlabel(r'$\log_{10}$(Period [days])')
plt.ylabel('Ks')
plt.legend()

Results are as espected, the fit performed with the linear fitter is better than the second one, non linear. 

**Conclusion:** Pay attention when you choose the fitter.

## 3) Fit a Gaussian: Lets compare to scipy

Maybe some of you know that scipy has the function [`scipy.optimize.curve_fit`](https://docs.scipy.org/doc/scipy-1.0.0/reference/generated/scipy.optimize.curve_fit.html) to fit in a similar way we are doing things. Lets compare the two methods with fake data in the shape of a gaussian.

In [ ]:
mu, sigma, amplitude = 0.0, 10.0, 10.0
N2 = 100
x2 = np.linspace(-30, 30, N)
y2 = amplitude * np.exp(-(x2-mu)**2 / (2*sigma**2))
y2 = np.array([y_point + np.random.normal(0, 1) for y_point in y2])
sigma = 1
y2_err = np.ones(N)*sigma

In [ ]:
plt.errorbar(x2, y2, yerr=y2_err, fmt='k.')
plt.xlabel('$x_2$')
plt.ylabel('$y_2$')

Lets do our three lines to make the fit we want. Note that we are not going to use the same linear fitter we were using, but one that is not linear (`LevMarLSQFitter`) because now our model requiers it. 

In [ ]:
model_gauss = models.Gaussian1D()
fitter_gauss = fitting.LevMarLSQFitter()
best_fit_gauss = fitter_gauss(model_gauss, x2, y2, weights=1/y2_err**2)

In [ ]:
print(best_fit_gauss)

To get the errors on the parameters, for this fitter, we can use `fitter.fit_info['param_cov']` which gives the [covariance matrix](http://mathworld.wolfram.com/CovarianceMatrix.html), and the elements in the diagonal are the errors. We can check the order of the parameters using:

In [ ]:
model_gauss.param_names

In [ ]:
cov_diag = np.diag(fitter_gauss.fit_info['param_cov'])
print(cov_diag)

Then:

In [ ]:
print('Amplitude: {} +\- {}'.format(best_fit_gauss.amplitude.value, np.sqrt(cov_diag[0])))
print('Mean: {} +\- {}'.format(best_fit_gauss.mean.value, np.sqrt(cov_diag[1])))
print('Standard Deviation: {} +\- {}'.format(best_fit_gauss.stddev.value, np.sqrt(cov_diag[2])))

Now lets do the same fit but using the method with scipy, and compare the results the same way we did before.

In [ ]:
import scipy

def f(x,a,b,c):
    return a * np.exp(-(x-b)**2/(2.0*c**2))

In [ ]:
p_opt, p_cov = scipy.optimize.curve_fit(f,x2, y2, sigma=y1_err)
a,b,c = p_opt
best_fit_gauss_2 = f(x2,a,b,c)

In [ ]:
print(p_opt)

In [ ]:
print('Amplitude: {} +\- {}'.format(p_opt[0], np.sqrt(p_cov[0,0])))
print('Mean: {} +\- {}'.format(p_opt[1], np.sqrt(p_cov[1,1])))
print('Standard Deviation: {} +\- {}'.format(p_opt[2], np.sqrt(p_cov[2,2])))

#### Compare results

In [ ]:
reduce_chi_squared = calc_reduce_chi_square(best_fit_gauss(x2), x2, y2, y2_err, N2, 3)
print('Reduce Chi Squared using astropy.modeling: {}'.format(reduce_chi_squared))

In [ ]:
reduce_chi_squared = calc_reduce_chi_square(best_fit_gauss_2, x2, y2, y2_err, N2, 3)
print('Reduce Chi Squared using scipy: {}'.format(reduce_chi_squared))

As we can see there is a very small difference in the *Reduce Chi Squared*, but the method with scipy require us to know the function we wanted to fit. In the case of `astropy.modeling`, if you check again the fits we made before, the only thing we changed to make different fits was the name of the model and the fitter.

In [ ]:
plt.errorbar(x2, y2, yerr=y2_err, fmt='k.')
plt.plot(x2, best_fit_gauss(x2), 'g-', linewidth=6, label='astropy.modeling')
plt.plot(x2, best_fit_gauss_2, 'r-', linewidth=2, label='scipy')
plt.xlabel('$x_2$')
plt.ylabel('$y_2$')
plt.legend()

**Conclusion:** Choose the method most convenient for every case you need to fit. We recomend astropy.modeling to make quick fits of known functions.

## 3) Exercise: Your turn to choose

Exercise: For the next data: 
* Choose model and fitter to fit this data.
* Compare different options.

In [ ]:
N3 = 100
x3 = np.linspace(0, 3, N3)
y3 = 5.0 * np.sin(2 * np.pi * x3)
y3 = np.array([y_point + np.random.normal(0, 1) for y_point in y3])
sigma = 1.5
y3_err = np.ones(N)*sigma 

In [ ]:
plt.errorbar(x3, y3, yerr=y3_err, fmt='k.')
plt.xlabel('$x_3$')
plt.ylabel('$y_3$')